In [14]:
import os
import pickle
import pandas as pd
import json
import numpy as np
from user_agents import parse
import unidecode
import re
import datetime
from tqdm import tqdm

from google.cloud import bigquery
from google.oauth2.service_account import Credentials

In [7]:
def read_json(file_path):
    with open(file_path, "r") as f:
        return json.load(f)

def select(engine,table_name,list_columns="*"):
    """ Select columns from a table """

    if list_columns=="*":
        query = f"SELECT * FROM {table_name}"

    else:
        query = '''
        SELECT "%s"
        FROM %s
        '''%('", "'.join(list_columns),table_name)

    query_result = pd.read_sql_query(query,engine)

    return query_result

def get_moment_from_time(time_,moments):

  order_moment = 'Nuit'

  for moment in moments:

    if moments[moment][0] <= time_ < moments[moment][1]:

      order_moment = moment
      
      break
  
  return moment

def get_device_from_ua(user_agent_string):
  user_agent = parse(user_agent_string)

  if user_agent.is_mobile:
    return('Mobile')
  elif user_agent.is_tablet:
    return('Tablette')
  elif user_agent.is_pc:
    return('PC')
  elif user_agent.is_bot:
    return('Bot')
  else:
    return('Indéterminé')

def get_discount_range(discount):

  discount_range = '100'
  try:
    for i in range(20):

      if discount >= i*5 and discount < (i+1)*5:
        discount_range = str(i*5)+" - "+str((i+1)*5)

    return discount_range
  except:
    return '0 - 5'

def dataframe_to_bigquery(dataframe, project_id, dataset, table_name, credential):

  dataframe_gbq = dataframe.copy()

  for col in dataframe_gbq.columns:
    new_col = col.replace(' ','_')
    new_col = new_col.replace("'",'_')
    new_col = unidecode.unidecode(new_col).lower()
    dataframe_gbq.rename(columns={col:new_col}, inplace=True)

  target_table = dataset+'.'+table_name

  dataframe_gbq.to_gbq(
      destination_table = target_table,
      project_id = project_id,
      if_exists = 'replace',
      progress_bar = True,
      credentials = credential
      )
  
  return 

def make_valeur_affichee_kpi(valeur, kpi, unites):

  if kpi in ['CA','Commandes','Clients']: # 0 arrondi
    valeur_str = ('{:,}'.format(round(valeur)).replace(',', ' ')).replace('.',',')
    return valeur_str+unites[kpi]

  elif kpi in ['Panier moyen']: # 1 arrondi
    valeur_str = ('{:,}'.format(round(valeur,1)).replace(',', ' ')).replace('.',',')
    return valeur_str+unites[kpi]

  elif kpi in ['Fréquence','Prix moyen article','Nb articles moyen']: # 2 arrondi
    valeur_str = ('{:,}'.format(round(valeur,2)).replace(',', ' ')).replace('.',',')
    return valeur_str+unites[kpi]

def make_var_affichee_kpi(var, kpi):

  if kpi in ['CA','Commandes','Clients']: 

    return str(round(var,1)).replace('.',',')+'% du total'

  elif kpi in ['Panier moyen','Prix moyen article','Nb articles moyen','Fréquence']:

    var_sign = '+' if var>0 else ''

    return var_sign+str(round(var,1)).replace('.',',')+'% vs moyenne'

  else:

    return ''


In [8]:
liste_kpis = [
    'CA',
    'Commandes',
    'Clients',
    'Panier moyen',
    'Prix moyen article',
    'Nb articles moyen',
    'Fréquence'
    ]

unites = {
    'CA':'€',
    'Commandes':'',
    'Clients':'',
    'Panier moyen':'€',
    'Prix moyen article':'€',
    'Nb articles moyen':'',
    'Fréquence':'x'
    }

equiv_demog_illu = {
    'genre':{
        'Femme':'F',
        'Homme':'H'
    },
    'age':{
        '18-24':'1829',    
        '25-34':'1829',
        '35-44':'3039',
        '45-54':'4049',
        '55-64':'5059',
        '65-74':'65',
        '75-99':'65'
    },'localisation':{
        'Très rural':'TR',
        'Rural':'R',
        'Urbain':'U',
        'Très urbain':'TU',
        'Inderterminé':'U',
        'Indéterminé':'U'
    },'revenus':{
        'CSP moins':'E',
        'CSP moyen':'EE',
        'CSP plus':'EEE'
    }
  }

moments = {
     'Matin': [datetime.time(7,0,0), datetime.time(10,00,0)],
     'Déjeuner': [datetime.time(11,0,0), datetime.time(15,00,0)],
     'Après-midi': [datetime.time(15,0,0), datetime.time(19,00,0)],
     'Soirée': [datetime.time(19,0,0), datetime.time(23,00,0)],
     'Nuit': [datetime.time(23,0,0), datetime.time(7,00,0)],
     }

dict_months = {
    1:'janvier',
    2:'février',
    3:'mars',
    4:'avril',
    5:'mai',
    6:'juin',
    7:'juillet',
    8:'août',
    9:'septembre',
    10:'octobre',
    11:'novembre',
    12:'décembre'
    }

dict_day = {
    'Monday':'Lundi',
    'Tuesday':'Mardi',
    'Wednesday':'Mercredi',
    'Thursday':'Jeudi',
    'Friday':'Vendredi',
    'Saturday':'Samedi',
    'Sunday':'Dimanche'
    }

## Connexion à BigQuery

In [2]:
###############      BigQuery Connection     ###############

project_id = 'feel-kaya-fr'
dataset = 'feelkayafr'

cred_paths = {
    'feel-kaya-fr':'keys/feelkayafr.json'
}

credentials_path = cred_paths[project_id]
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
os.environ['PROJECT-ID'] = project_id

client = bigquery.Client()

credential = Credentials.from_service_account_file(credentials_path)

In [3]:
### CUSTOMERS PROFILES

query = f"""
        SELECT *
        FROM {project_id}.{dataset}.customer_profiles
    """

customer_profiles = client.query(query).to_dataframe()
customer_profiles['splitted'] = customer_profiles['id'].apply(lambda x: x.split('_'))
customer_profiles['customer_id'] = customer_profiles['splitted'].apply(lambda x: x[0])
customer_profiles['first_demographic'] = customer_profiles['splitted'].apply(lambda x: x[1])
customer_profiles['len(splitted)'] = customer_profiles['splitted'].apply(lambda x: len(x))

cross_customer_profiles = customer_profiles.loc[customer_profiles['len(splitted)']==8].copy()
cross_customer_profiles.drop(['first_demographic','len(splitted)'], axis=1, inplace=True)
simple_customer_profiles = customer_profiles.loc[customer_profiles['len(splitted)']!=8].copy()

localisation = simple_customer_profiles.loc[simple_customer_profiles['first_demographic']=='localisation']
localisation = localisation[['customer_id','probabilite']]
localisation.rename({'probabilite':'localisation'}, axis=1, inplace=True)
localisation.set_index('customer_id', inplace=True)
localisation.replace({0:"Indéterminé",1:"Très rural",2:"Rural",3:"Urbain",4:"Très urbain"}, inplace=True)

cross = cross_customer_profiles[['id','probabilite']].copy()
cross['cross_id'] = cross['id'].apply(lambda x: x[:-10] if len(x)==41 else x[:-11])
cross['D'] = cross['id'].apply(lambda x: x[-2:] if len(x)==41 else x[-3:] )
cross = pd.pivot_table(cross, values=['probabilite'], index=['cross_id'], columns=['D'])
cross.columns = ['D1','D10','D2','D3','D4','D5','D6','D7','D8','D9']

cross['CSP moins'] = cross['D1']     + cross['D2'] + cross['D3'] + cross['D4']/3
cross['CSP moyen'] = cross['D4']*2/3 + cross['D5'] + cross['D6'] + cross['D7']*2/3
cross['CSP plus']  = cross['D7']/3   + cross['D8'] + cross['D9'] + cross['D10']
cross['S'] = cross['CSP moins'] + cross['CSP moyen'] + cross['CSP plus']

cross['genre'] = cross.index.map(lambda x: x.split('_')[2])
cross['age'] = cross.index.map(lambda x: x[-5:])
cross['customer_id'] = cross.index.map(lambda x: x[:13])

cross_light = cross[['CSP moins','CSP moyen','CSP plus','genre','age','customer_id']]
cross_light_customer_id = cross_light.set_index('customer_id', drop=False)
cross_light_customer_id = cross_light_customer_id.join(localisation)

cross_light_customer_id['genre'].replace({'F':'Femme','M':'Homme'}, inplace=True)
cross_light_customer_id['age'] = cross_light_customer_id['age'].apply(lambda x : x.replace('_','-'))

melted_cross_light_customer_id = pd.melt(cross_light_customer_id, id_vars=['customer_id', 'age', 'genre', 'localisation'], value_vars=['CSP moins', 'CSP moyen', 'CSP plus'])
melted_cross_light_customer_id.rename(columns = {'variable':'revenus'}, inplace=True)
melted_cross_light_customer_id.set_index('customer_id', inplace=True)

with open('data/melted_customer_id.pickle', 'wb') as f:
  pickle.dump(melted_cross_light_customer_id, f)

In [ ]:
### ORDERS

query = f"""
    SELECT *
    FROM {project_id}.{dataset}.orders_cleaned
"""

orders = client.query(query).to_dataframe()
orders.set_index('customer_id', inplace=True)

total_nb_orders = len(orders)
total_nb_clients = len(orders.index.unique())

orders = orders.loc[melted_cross_light_customer_id.index.unique()]

final_nb_orders = len(orders)
final_nb_clients = len(orders.index.unique())

print('Nb total commandes :', total_nb_orders, '| Nb total clients :', total_nb_clients)
print('Nb total commandes :', final_nb_orders, '| Nb total clients :', final_nb_clients)
print('% commandes :', round(100*final_nb_orders/total_nb_orders,2), '% | % clients', round(100*final_nb_clients/total_nb_clients,2), '%' )

# discount range of order
orders['discount'] =  orders['discount_percent'].apply( lambda x: get_discount_range(100*x) )

# moment of order
orders['moment'] = orders['time'].apply(lambda x: get_moment_from_time(x,moments))

# device used
orders['device'] = orders['user_agent'].apply(lambda x: get_device_from_ua(x) if x else 'Indéterminé' )

In [19]:
orders = orders.loc[orders['processed_at'].apply(lambda x: x.year==2021)].copy()

orders.loc['blank'] = ''

res_orders = {}

####   Calcul des kpis pour tous les customers ID sur l'année 2021

for customer_id in set(orders.index): 

  if customer_id != 'blank':

    r_customer = {}

    customer_orders = orders.loc[[customer_id,'blank']]
    customer_orders = customer_orders.drop('blank')

    # CA
    r_customer['CA'] = customer_orders['price'].sum()

    # Commandes
    r_customer['Commandes'] = len(customer_orders)

    # Panier moyen
    r_customer['Panier moyen'] = r_customer['CA'] / r_customer['Commandes']

    nb_articles = customer_orders['nb_items'].sum()

    # Prix moyen article
    r_customer['Prix moyen article'] = r_customer['CA'] / nb_articles

    # Nombre moyen articles
    r_customer['Nb articles moyen'] = nb_articles / r_customer['Commandes']

    # Clients
    r_customer['Clients'] = 1

    # Fréquence
    r_customer['Fréquence'] = r_customer['Commandes']

    res_orders[customer_id] = r_customer

orders.drop('blank', inplace=True)

df_customers_kpis = pd.DataFrame.from_dict(res_orders,orient='index')

#####  Calcul des kpis globaux sur l'année 2021

total_kpi = {}

for col in df_customers_kpis.columns :

  if col in ['CA','Commandes','Clients']:
    total_kpi[col] = df_customers_kpis[col].sum()
  elif col in ['Panier moyen','Prix moyen article','Nb articles moyen','Fréquence']:
    total_kpi[col] = np.average(df_customers_kpis[col], weights=df_customers_kpis['Commandes'])

#### CA total par mois

months = [m for m in range(1,13)]

ca_total_year = {}

for month in tqdm(months):
  
  orders_month = orders.loc[orders['month'] == month].copy()

  ca_total_year[month] = orders_month['price'].sum()

with open('data/customer_kpis.pickle', 'wb') as f:
  pickle.dump(df_customers_kpis, f)

100%|██████████| 12/12 [00:00<00:00, 553.35it/s]


In [79]:
orders = orders.loc[orders['processed_at'].apply(lambda x: x.year==2021)].copy()

orders.loc['blank'] = ''

res_orders = {}

####   Calcul des kpis pour tous les customers ID sur l'année 2021

for customer_id in set(orders.index): 

  if customer_id != 'blank':

    r_customer = {}

    customer_orders = orders.loc[[customer_id,'blank']]
    customer_orders = customer_orders.drop('blank')

    # CA
    r_customer['CA'] = customer_orders['price'].sum()

    # Commandes
    r_customer['Commandes'] = len(customer_orders)

    # Panier moyen
    r_customer['Panier moyen'] = r_customer['CA'] / r_customer['Commandes']

    nb_articles = customer_orders['nb_items'].sum()

    # Prix moyen article
    r_customer['Prix moyen article'] = r_customer['CA'] / nb_articles

    # Nombre moyen articles
    r_customer['Nb articles moyen'] = nb_articles / r_customer['Commandes']

    # Clients
    r_customer['Clients'] = 1

    # Fréquence
    r_customer['Fréquence'] = r_customer['Commandes']

    res_orders[customer_id] = r_customer

orders.drop('blank', inplace=True)

df_orders_kpi = pd.DataFrame.from_dict(res_orders,orient='index')

#####  Calcul des kpis globaux sur l'année 2021

total_kpi = {}

for col in df_orders_kpi.columns :

  if col in ['CA','Commandes','Clients']:
    total_kpi[col] = df_orders_kpi[col].sum()
  elif col in ['Panier moyen','Prix moyen article','Nb articles moyen','Fréquence']:
    total_kpi[col] = np.average(df_orders_kpi[col], weights=df_orders_kpi['Commandes'])

#### CA total par mois

months = [m for m in range(1,13)]

ca_total_year = {}

for month in tqdm(months):
  
  orders_month = orders.loc[orders['month'] == month].copy()

  ca_total_year[month] = orders_month['price'].sum()

100%|██████████| 12/12 [00:00<00:00, 486.02it/s]


In [80]:
####   Calcul des kpis pour tous les croisements age x genre x localisation x revenus

df_kpi_merged = pd.merge(left=melted_cross_light_customer_id, right=df_orders_kpi, left_index=True, right_index=True, how='right')

for kpi in df_orders_kpi.columns:
  df_kpi_merged[kpi] = df_kpi_merged[kpi]*df_kpi_merged['value']

total_res = {}

for genre in df_kpi_merged['genre'].unique():

  for age in df_kpi_merged['age'].unique():

    for localisation in df_kpi_merged['localisation'].unique():
    
      for revenu in df_kpi_merged['revenus'].unique():

        profile_res = {}

        profile_res['age'] = age
        profile_res['genre'] = genre
        profile_res['localisation'] = localisation
        profile_res['revenus'] = revenu

        df_profile = df_kpi_merged.loc[(df_kpi_merged['age']==age) & (df_kpi_merged['genre']==genre) & (df_kpi_merged['localisation']==localisation) & (df_kpi_merged['revenus']==revenu)]
        
        for kpi in ['CA','Commandes','Clients']:
          profile_res[kpi] = df_profile[kpi].sum()

        for kpi in ['Panier moyen','Prix moyen article','Nb articles moyen', 'Fréquence']:
          profile_res[kpi] = (df_profile[kpi]*df_profile['Commandes']).sum() / (df_profile['value']*df_profile['Commandes']).sum()

        total_res[genre+'_'+age+'_'+localisation+'_'+revenu] = profile_res

df_total_kpis = pd.DataFrame.from_dict(total_res, orient='index')

## BDD Analytique

In [27]:
# imports

import pandas as pd
import json
from sqlalchemy import create_engine, engine
from sqlalchemy_utils import database_exists, create_database

# modules
def read_json(file_path):
    with open(file_path, "r") as f:
        return json.load(f)

# test
def test_db_config(db_config):
    """
    Test if the database config is valid
    """

    s = f"postgresql://{db_config['user']}:{db_config['pass']}@{db_config['host']}:{db_config['port']}/postgres"
    db_engine = create_engine(s)

    assert type(db_engine.connect()) == engine.base.Connection

In [37]:
# read config
db_config = read_json("keys/database_config.json")

# test database connection with config
test_db_config(db_config)
print("- Connection to db created successfully with current config")

# Create engine. Make sure the current IP address is withelisted on Azure.
s = f"postgresql://{db_config['user']}:{db_config['pass']}@{db_config['host']}:{db_config['port']}/an_database"
an_engine = create_engine(s)

- Connection to db created successfully with current config


In [ ]:
### Create database if it doesn't exist
# Create tables
s = """ 

DROP TABLE IF EXISTS profiles CASCADE;
CREATE TABLE profiles (
    profile_id SERIAL PRIMARY KEY,
    genre VARCHAR(50) NOT NULL,
    age VARCHAR(50) NOT NULL,
    revenus VARCHAR(50) NOT NULL,
    localisation VARCHAR(50) NOT NULL
);

DROP TABLE IF EXISTS customers CASCADE;
CREATE TABLE customers (
    customer_id VARCHAR(50) PRIMARY KEY
);

DROP TABLE IF EXISTS customer_profiles CASCADE;
CREATE TABLE customer_profiles (
    profile_id SERIAL REFERENCES profiles (profile_id),
    customer_id VARCHAR(50) REFERENCES customers (customer_id),
    value FLOAT NOT NULL,
    PRIMARY KEY (profile_id, customer_id)
);

DROP TABLE IF EXISTS orders CASCADE;
CREATE TABLE orders (
    order_id VARCHAR(50) PRIMARY KEY,
    customer_id VARCHAR(50),
    price FLOAT NOT NULL,
    discount_amount	FLOAT NOT NULL,
    discount_percent FLOAT NOT NULL,
    firstname VARCHAR(50) NOT NULL, 
    billing_address JSON NOT NULL,
    processed_at TIMESTAMP NOT NULL,
    time TIME NOT NULL,
    day VARCHAR(50) NOT NULL,
    month VARCHAR(50) NOT NULL,
    user_agent VARCHAR NOT NULL,
    items JSON NOT NULL,
    nb_items INT NOT NULL,
    discount VARCHAR(50) NOT NULL,
    moment VARCHAR(50) NOT NULL,
    device VARCHAR(50) NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers (customer_id) ON UPDATE CASCADE ON DELETE CASCADE 
);

DROP TABLE IF EXISTS profiles_kpis CASCADE;
CREATE TABLE profiles_kpis (
    profile_id SERIAL NOT NULL,
    ca FLOAT NOT NULL,
    commandes FLOAT NOT NULL,
    panier_moyen FLOAT NOT NULL,
    prix_moyen_article FLOAT NOT NULL,
    clients FLOAT NOT NULL,
    nb_articles_moyen FLOAT NOT NULL,
    frequence FLOAT NOT NULL,
    FOREIGN KEY (profile_id) REFERENCES profiles (profile_id) ON UPDATE CASCADE ON DELETE CASCADE 
)
"""

an_engine.execute(s)

# Customers
customers = melted_cross_light_customer_id.reset_index().loc[:,'customer_id'].drop_duplicates()

customers.to_sql('customers',an_engine,if_exists='append',index=False)

# Orders
orders_to_sql = orders.copy()

for col in ['billing_address','items']:
    orders_to_sql[col] = orders_to_sql[col].apply(lambda x: json.dumps(str(x)))

orders_to_sql.to_sql('orders',an_engine,if_exists='append',index=True)

# Profiles
profiles = melted_cross_light_customer_id.copy().groupby(['genre','age','revenus','localisation']).count().reset_index()
profiles = profiles[['genre','age','revenus','localisation']]
profiles['profile_id'] = profiles.index+1

profiles.to_sql('profiles',an_engine,if_exists='append',index=False)

# Customer profiles
profiles_ids = profiles.set_index(['genre','age','revenus','localisation'])

customer_profiles = melted_cross_light_customer_id.copy().reset_index()
customer_profiles['profile_id'] = customer_profiles.apply( lambda row : profiles_ids.loc[ (row['genre'], row['age'], row['revenus'], row['localisation']), 'profile_id' ] , axis=1 )
customer_profiles.rename(columns={'valeur':'value'},inplace=True)

customer_profiles[['customer_id','profile_id','value']].to_sql('customer_profiles',an_engine,if_exists='append',index=False)

# Profiles kpis
profiles_kpis = df_total_kpis.copy()

profiles_kpis['profile_id'] = profiles_kpis.apply( lambda row : profiles_ids.loc[ (row['genre'], row['age'], row['revenus'], row['localisation']), 'profile_id' ] , axis=1 )
profiles_kpis.rename(columns={
    'CA':'ca',
    'Commandes':'commandes',
    'Panier moyen':'panier_moyen',
    'Prix moyen article':'prix_moyen_article',
    'Clients':'clients',
    'Nb articles moyen':'nb_articles_moyen',
    'Fréquence':'frequence'
}, inplace=True)
profiles_kpis = profiles_kpis[
        [
            'profile_id',
            'ca',
            'commandes',
            'panier_moyen',
            'prix_moyen_article',
            'clients',
            'nb_articles_moyen',
            'frequence'
        ]
    ]

profiles_kpis.to_sql('profiles_kpis',an_engine,if_exists='append',index=False)

In [140]:
query = """
    SELECT * FROM profiles
    JOIN profiles_kpis ON profiles.profile_id = profiles_kpis.profile_id
"""

df_profiles = pd.read_sql(query,an_engine)
df_profiles.drop('profile_id',axis=1,inplace=True)
df_profiles.head(20)

,genre,age,revenus,localisation,ca,commandes,panier_moyen,prix_moyen_article,clients,nb_articles_moyen,frequence
0,Femme,18-24,CSP moins,Très urbain,51503.269416,928.496894,54.143276,40.431653,750.146705,1.415168,1.619840
1,Femme,18-24,CSP moyen,Très urbain,26462.056050,477.813439,53.812686,40.004087,385.711183,1.419154,1.650473
2,Femme,18-24,CSP plus,Très urbain,35576.056214,640.514657,53.992378,40.499329,517.355698,1.399364,1.630459
3,Femme,18-24,CSP moins,Très rural,14030.376582,245.241309,55.218403,42.111052,201.661046,1.398460,1.536289
4,Femme,18-24,CSP moyen,Très rural,10738.769284,186.788709,56.402980,42.641786,154.486127,1.411537,1.459353
5,Femme,18-24,CSP plus,Très rural,8143.977506,139.798292,58.124413,43.201481,115.673010,1.425455,1.485414
6,Femme,18-24,CSP moins,Urbain,24425.430127,439.566736,53.994363,41.233375,362.903372,1.402084,1.465628
7,Femme,18-24,CSP moyen,Urbain,16003.268180,287.305282,54.757529,41.308634,235.059210,1.400475,1.547464
8,Femme,18-24,CSP plus,Urbain,14879.592955,269.328511,52.941337,39.869903,220.463472,1.390395,1.536217
9,Femme,18-24,CSP moins,Rural,20330.076240,351.119826,56.458179,42.826173,276.241445,1.398191,1.940766
